# 《动手学 AI 01：初识机器学习》

课程目标：从零构建第一个AI模型，快速掌握机器学习的本质，理解机器是何以学习的。

In [1]:
from util import show_version, print_table, print_regression_results

show_version()

Python version:  3.10.15 (main, Oct  3 2024, 02:24:49) [Clang 14.0.6 ]
Pytorch version:  2.7.0
PyTorch Lightning version:  2.5.1


## 一、程序思维

In [2]:
# 使用程序计算放糖量

def col_sugar(lemon):
    if lemon == 1:
        return "10"
    elif lemon == 2:
        return "20"
    else:
        return "没有此配方，做不了❗️"


print_table("用程序制作柠檬水", ["柠檬片（输入）", "放糖量（输出）"],
            [["1 片", col_sugar(1)], ["2 片", col_sugar(2)],
             ["3 片", col_sugar(3)], ["4 片", col_sugar(4)],
             ["5 片", col_sugar(4)]])


用程序制作柠檬水：
+----------------+----------------------+
| 柠檬片（输入） |    放糖量（输出）    |
+----------------+----------------------+
|      1 片      |          10          |
|      2 片      |          20          |
|      3 片      | 没有此配方，做不了❗️ |
|      4 片      | 没有此配方，做不了❗️ |
|      5 片      | 没有此配方，做不了❗️ |
+----------------+----------------------+


## 二、准备数据


In [3]:
# 专家的示范
lemon = 2
sugar = 16

print_table(
    f"训练数据",
    ["柠檬片（输入数据）", "放糖量（结果标签）"],
    [[lemon, sugar]]
)


训练数据：
+--------------------+--------------------+
| 柠檬片（输入数据） | 放糖量（结果标签） |
+--------------------+--------------------+
|         2          |         16         |
+--------------------+--------------------+


## 三、模型定义

In [4]:
import random
import pickle


class Model:

    def __init__(self):
        # 随机初始化糖权
        # self.sugar_weight = random.uniform(0, 50)
        self.sugar_weight = 1.88
        print(f"🚀随机初始化的糖权: {self.sugar_weight}")

    # 1️⃣前向传播，计算预测值
    def forward(self, lemon):
        return self.sugar_weight * lemon

    # 2️⃣计算误差损失
    def loss(self, pred_sugar, real_sugar):
        return pred_sugar - real_sugar

    # 3️⃣计算更新参数的方向和步长
    def backward(self, loss):
        if loss > 0:
            # 如果损失值大于0，表示预测值偏大
            return -0.1
        elif loss < 0:
            # 如果损失值小于0，表示预测值偏小
            return 0.1

    # 4️⃣更新糖权
    def step(self, step):
        self.sugar_weight += step

## 四、模型训练

用训练数据对模型进行训练，通过优化算法调整参数，最小化损失函数（预测和实际结果之间的差距）。

In [5]:
# 训练模型

# 创建模型实例
model = Model()

training_logs = []

# 训练模型
for epoch in range(100):
    # 1. 计算预测值
    pred = model.forward(lemon)
    # 2. 计算损失
    loss = model.loss(pred, sugar)
    # 3. 计算更新的步长（梯度）
    step = model.backward(loss)
    # 4. 更新参数
    model.step(step)

    # 记录训练日志，包括更新之后的权重（保留四位小数）
    # 按照表头顺序调整：Epoch, 柠檬盘, 当前糖权, 预测放糖量, 真实放糖量, 误差损失, 更新步长, 更新后权重
    training_logs.append([
        epoch,
        lemon,
        round(model.sugar_weight - step, 4),
        round(pred, 4),
        sugar,
        round(loss, 4),
        round(step, 4),
        round(model.sugar_weight, 4)
    ])

# 打印最终参数
print(f"✅模型学习到的糖权: {round(model.sugar_weight, 4)}")

print_table("📝训练日志",
            ["Epoch", "柠檬片", "当前的糖权", "预测放糖量", "真实放糖量", "误差损失", "更新步长", "更新后糖权"],
            training_logs)

🚀随机初始化的糖权: 1.88
✅模型学习到的糖权: 8.08

📝训练日志：
+-------+--------+------------+------------+------------+----------+----------+------------+
| Epoch | 柠檬片 | 当前的糖权 | 预测放糖量 | 真实放糖量 | 误差损失 | 更新步长 | 更新后糖权 |
+-------+--------+------------+------------+------------+----------+----------+------------+
|   0   |   2    |    1.88    |    3.76    |     16     |  -12.24  |   0.1    |    1.98    |
|   1   |   2    |    1.98    |    3.96    |     16     |  -12.04  |   0.1    |    2.08    |
|   2   |   2    |    2.08    |    4.16    |     16     |  -11.84  |   0.1    |    2.18    |
|   3   |   2    |    2.18    |    4.36    |     16     |  -11.64  |   0.1    |    2.28    |
|   4   |   2    |    2.28    |    4.56    |     16     |  -11.44  |   0.1    |    2.38    |
|   5   |   2    |    2.38    |    4.76    |     16     |  -11.24  |   0.1    |    2.48    |
|   6   |   2    |    2.48    |    4.96    |     16     |  -11.04  |   0.1    |    2.58    |
|   7   |   2    |    2.58    |    5.16    |     16     |  -1

## 五、模型改进

存在的问题：
1. 学习速度慢
2. 学习存在震荡

改进思路：使用动态学习率，差得多学的多，差的少学的少。

### 4.1 改进模型

In [6]:
import random
import pickle


class Model:

    def __init__(self):
        # 随机初始化糖权
        # self.sugar_weight = random.uniform(0, 50)
        self.sugar_weight = 1.88
        print(f"🚀随机初始化的糖权: {self.sugar_weight}")

    # 1️⃣前向传播，计算预测值
    def forward(self, lemon):
        return self.sugar_weight * lemon

    # 2️⃣计算误差损失
    def loss(self, pred_sugar, real_sugar):
        return pred_sugar - real_sugar

    # 3️⃣计算更新参数的方向和步长
    def backward(self, loss):
        return -loss

    # 4️⃣更新糖权
    def step(self, step, learning_rate):
        self.sugar_weight += step * learning_rate

### 4.2 改进训练过程

In [7]:
# 训练模型

# 创建模型实例
model = Model()

training_logs = []

learning_rate = 0.1

# 训练模型
for epoch in range(100):
    # 1. 计算预测值
    pred = model.forward(lemon)
    # 2. 计算损失
    loss = model.loss(pred, sugar)
    # 3. 计算更新的步长（梯度）
    step = model.backward(loss)
    # 4. 更新参数
    model.step(step, learning_rate)

    # 记录训练日志，包括更新之后的权重（保留四位小数）
    # 按照表头顺序调整：Epoch, 柠檬盘, 当前糖权, 预测放糖量, 真实放糖量, 误差损失, 更新步长, 更新后权重
    training_logs.append([
        epoch,
        lemon,
        round(model.sugar_weight - step * learning_rate, 4),
        round(pred, 4),
        sugar,
        round(loss, 4),
        round(step, 4),
        round(model.sugar_weight, 4)
    ])

# 打印最终参数
print(f"✅模型学习到的糖权: {round(model.sugar_weight, 4)}")

print_table("📝训练日志",
            ["Epoch", "柠檬片", "当前的糖权", "预测放糖量", "真实放糖量", "误差损失", "更新步长", "更新后糖权"],
            training_logs)

🚀随机初始化的糖权: 1.88
✅模型学习到的糖权: 8.0

📝训练日志：
+-------+--------+------------+------------+------------+----------+----------+------------+
| Epoch | 柠檬片 | 当前的糖权 | 预测放糖量 | 真实放糖量 | 误差损失 | 更新步长 | 更新后糖权 |
+-------+--------+------------+------------+------------+----------+----------+------------+
|   0   |   2    |    1.88    |    3.76    |     16     |  -12.24  |  12.24   |   3.104    |
|   1   |   2    |   3.104    |   6.208    |     16     |  -9.792  |  9.792   |   4.0832   |
|   2   |   2    |   4.0832   |   8.1664   |     16     | -7.8336  |  7.8336  |   4.8666   |
|   3   |   2    |   4.8666   |   9.7331   |     16     | -6.2669  |  6.2669  |   5.4932   |
|   4   |   2    |   5.4932   |  10.9865   |     16     | -5.0135  |  5.0135  |   5.9946   |
|   5   |   2    |   5.9946   |  11.9892   |     16     | -4.0108  |  4.0108  |   6.3957   |
|   6   |   2    |   6.3957   |  12.7914   |     16     | -3.2086  |  3.2086  |   6.7165   |
|   7   |   2    |   6.7165   |  13.4331   |     16     | -2.5

## 六、模型预测

In [8]:
# 使用预训练的模型进行预测未见过的样本
lemons = [3, 4, 5]
real_sugars = [24, 32, 40]
pred_sugars = [model.forward(lemon) for lemon in lemons]

print_regression_results(lemons, real_sugars, pred_sugars, "制作新的柠檬水",
                         ["柠檬片", "真实放糖量", "预测放糖量", "预测偏差", "预测精度"])


制作新的柠檬水：
+--------+------------+------------+----------+----------+
| 柠檬片 | 真实放糖量 | 预测放糖量 | 预测偏差 | 预测精度 |
+--------+------------+------------+----------+----------+
|   3    |     24     |    24.0    | -0.0000  | 100.00%  |
|   4    |     32     |    32.0    | -0.0000  | 100.00%  |
|   5    |     40     |    40.0    | -0.0000  | 100.00%  |
+--------+------------+------------+----------+----------+


## 七、模型保存与加载

In [9]:
def save_parameters(model: Model, file_path):
    # 保存模型参数
    with open(file_path, "wb") as f:
        pickle.dump(model.sugar_weight, f)
    print(f"模型参数保存到 {file_path}")


def load_parameters(model: Model, file_path):
    # 加载模型参数
    with open(file_path, "rb") as f:
        model.sugar_weight = pickle.load(f)
    print(f"从 {file_path} 加载模型参数")

In [10]:
# 保存模型参数
save_parameters(model, "model.pkl")

模型参数保存到 model.pkl


In [11]:
# 加载训练好的模型
model = Model()
load_parameters(model, "model.pkl")
print(f"✅加载学习到的糖权: {model.sugar_weight:.4f}")

🚀随机初始化的糖权: 1.88
从 model.pkl 加载模型参数
✅加载学习到的糖权: 8.0000


In [12]:
# 使用预训练的模型进行预测未见过的样本
lemons = [3, 4, 5]
real_sugars = [24, 32, 40]
pred_sugars = [model.forward(lemon) for lemon in lemons]

print_regression_results(lemons, real_sugars, pred_sugars, "制作新的柠檬水",
                         ["柠檬片", "真实放糖量", "预测放糖量", "预测偏差", "预测精度"])


制作新的柠檬水：
+--------+------------+------------+----------+----------+
| 柠檬片 | 真实放糖量 | 预测放糖量 | 预测偏差 | 预测精度 |
+--------+------------+------------+----------+----------+
|   3    |     24     |    24.0    | -0.0000  | 100.00%  |
|   4    |     32     |    32.0    | -0.0000  | 100.00%  |
|   5    |     40     |    40.0    | -0.0000  | 100.00%  |
+--------+------------+------------+----------+----------+
